In [1]:
#5.23

import cv2
import mediapipe as mp
import csv
from datetime import datetime, timedelta
import math
import numpy as np

#CSV 파일에 헤더를 추가

with open('0.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Mouth Brightness', 'Mouth Openness', 'Mouth Width', 'Label'])

# Mediapipe의 Face Mesh 모델 로드
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# 웹캠에서 영상을 받아오는 VideoCapture 객체 생성
cap = cv2.VideoCapture(0)

start_time = datetime.now()
elapsed_time = timedelta(seconds=0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("비디오 캡처 실패, 종료합니다.")
        break
    
    # 프레임을 BGR에서 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 얼굴 검출 및 랜드마크 추출
    results = face_mesh.process(rgb_frame)

    mouth_indices = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291]  # 입 윤곽선을 정의하는 인덱스들
    
    if results.multi_face_landmarks:
     
        for face_landmarks in results.multi_face_landmarks:
            mouth_points = []
            for idx in mouth_indices:
                landmark = face_landmarks.landmark[idx]
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                mouth_points.append((x, y))
                # 랜드마크 위치에 작은 원 그리기
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
                # 랜드마크 인덱스 텍스트로 표시
                cv2.putText(frame, str(idx), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)
            
         
           
   
            
            # 입의 넓이(면적) 계산
            mouth_contour = np.array(mouth_points, dtype=np.int32)
            mouth_area = cv2.contourArea(mouth_contour)
            cv2.putText(frame, f'Mouth Area: {mouth_area:.2f}', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


            mouth_landmarks = face_landmarks.landmark[12:20]
            
            # 전체 프레임의 평균 밝기 계산
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            overall_brightness = frame_gray.mean()
            
            # 입 주변 영역의 명암도 계산을 위한 마스크 생성
            mask = np.zeros(frame_gray.shape, dtype=np.uint8)
            mouth_points = []
            for landmark in mouth_landmarks:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                mouth_points.append((x, y))
            cv2.fillPoly(mask, [np.array(mouth_points, dtype=np.int32)], 255)
            
            # 입 주변 영역의 명암도 계산
            mouth_area_gray = cv2.bitwise_and(frame_gray, frame_gray, mask=mask)
            mouth_brightness = cv2.mean(mouth_area_gray, mask=mask)[0]
            
            # 입 주변 명암도를 전체 밝기를 기준으로 상대값으로 변환
            relative_mouth_brightness = mouth_brightness / overall_brightness
            
            # 입 주변의 랜드마크 표시
            for landmark in mouth_landmarks:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 2, (255, 0, 0), -1)

                #입의 세로길이=>고중저모음 판단기준

                upper_lip_bottom = (face_landmarks.landmark[13].x, face_landmarks.landmark[13].y)
                lower_lip_top = (face_landmarks.landmark[14].x, face_landmarks.landmark[14].y)
                mouth_openness = lower_lip_top[1] - upper_lip_bottom[1]
            
                # 입의 가로길이를 계산=>원순/평순 판단기준
                lip_left =(face_landmarks.landmark[61].x, face_landmarks.landmark[61].y)
                lip_right=(face_landmarks.landmark[291].x, face_landmarks.landmark[291].y)
                mouth_width=lip_right[0]-lip_left[0]
                
            

            
            # 경과 시간이 0.2초 이상일 때만 CSV 파일에 추가
            if elapsed_time.total_seconds() >= 0.2:
                # 시간 단위로 입안 명암도, 입 벌림 정도, 입의 가로길이를 CSV 파일에 추가
                with open('0.csv', mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([relative_mouth_brightness,mouth_openness,mouth_width]) #변경사항
                
                # 경과 시간 초기화
                elapsed_time = timedelta(seconds=0)
            else:
                # 경과 시간 누적
                elapsed_time += datetime.now() - start_time
                start_time = datetime.now()
            
            # 입 벌림 정도, 입안 명암도, 입의 가로길이를 프레임에 표시
            cv2.putText(frame, f"Mouth Openness: {mouth_openness:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Mouth Brightness: {relative_mouth_brightness:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Mouth Width: {mouth_width:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
    # 영상 출력
    cv2.imshow('Face Mesh', frame)
    
    # 종료 키 (q) 입력 시 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 

# 자원 해제
cap.release()
cv2.destroyAllWindows()
